In [1]:
from pathlib import Path

import torch
from tqdm import tqdm

In [3]:
SAMPLE_RATE = 48000


# torch.set_num_threads(1)

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=False)

(_,  # get_speech_ts
 get_speech_ts_adaptive,
 save_audio,
 read_audio,
 _,  # state_generator
 _,  # single_audio_stream
 collect_chunks) = utils

Downloading: "https://github.com/snakers4/silero-vad/archive/master.zip" to /home/uadmin/.cache/torch/hub/master.zip


In [4]:
path = Path('data/audio/raw')
processed_path = Path('data/audio/processed')
processed_path.mkdir(exist_ok=True)

filepath_list = list(path.rglob('*.flac'))
print(f'Number of audio files found: {len(filepath_list)}')
print('Performing pausation cutting...')

for file in tqdm(filepath_list):
     wave_tensor = read_audio(file, target_sr=SAMPLE_RATE)
     speech_timestamps = get_speech_ts_adaptive(wave_tensor, model)
     speaker_dir = processed_path / file.parent.name
     speaker_dir.mkdir(exist_ok=True)
     save_audio(
          speaker_dir / file.name,
          collect_chunks(speech_timestamps, wave_tensor),
          SAMPLE_RATE
     )

  0%|          | 0/704 [00:00<?, ?it/s]

Number of audio files found: 704
Performing pausation cutting...


/home/uadmin/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:889: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:639.)
  result = self.forward(*input, **kwargs)
100%|██████████| 704/704 [01:11<00:00,  9.89it/s]
